# Load, Fuel Mix, and LMP Data

This notebook walk through how to use `gridstatus` to access to the data availabe on [OASIS](http://oasis.caiso.com/).

While we will be using CAISO in this example, most of this API will also work with all other ISOs.

In [2]:
import gridstatus
import pandas as pd
import os

os.chdir(r"C:\Users\gotta\OneDrive\Documents\Bureau\X\4A\US\Stanford\Classes\CS 230\Project\CS-230-Deep-Learning-Project")


In [ ]:
def download_market_data(market_name, start, end):
    """
    Download hourly renewable mix (each renewable), hourly load,
    and hourly average LMP over all locations for a given market.
    Save each as a CSV file.
    Supported markets: 'CAISO', 'PJM', 'NYISO'
    """
    market_map = {
        "CAISO": gridstatus.CAISO,
        "PJM": gridstatus.PJM,
        "NYISO": gridstatus.NYISO,
    }
    if market_name not in market_map:
        raise ValueError("Market must be one of: CAISO, PJM, NYISO")
    market = market_map[market_name]()

    # Fuel mix (renewables only, hourly evolution)
    mix_df = market.get_fuel_mix(start, end=end, verbose=False)
    mix_df['Time'] = pd.to_datetime(mix_df['Time'])
    renewables = ['Solar', 'Wind', 'Large Hydro', 'Small Hydro', 'Geothermal', 'Hydro', 'Other']
    renew_cols = [col for col in renewables if col in mix_df.columns]
    renew_mix = mix_df[['Time'] + renew_cols].set_index('Time').resample('h').mean().reset_index()
    renew_mix.to_csv(rf"data\US Markets\{market_name}\renewable_mix.csv", index=False)

    # Load (hourly)
    load_df = market.get_load(start, end=end)
    load_df['Time'] = pd.to_datetime(load_df['Time'])
    hourly_load = load_df.set_index('Time').resample('h').mean().reset_index()
    hourly_load.to_csv(rf"data\US Markets\{market_name}\load.csv", index=False)

    # LMP (hourly average over all locations)
    lmp_df = market.get_lmp(start=start, end=end, market="DAY_AHEAD_HOURLY")
    lmp_df['Time'] = pd.to_datetime(lmp_df['Time'])
    hourly_avg_lmp = (
        lmp_df.groupby('Time')['LMP'].mean().reset_index().rename(columns={'LMP': 'Avg_LMP'})
    )
    hourly_avg_lmp.to_csv(rf"data\US Markets\{market_name}\hourly_avg_lmp.csv", index=False)

    print("Saved CSVs:")
    print(f"- data\\US Markets\\{market_name}\\renewable_mix.csv")
    print(f"- data\\US Markets\\{market_name}\\load.csv")
    print(f"- data\\US Markets\\{market_name}\\hourly_avg_lmp.csv")

# Example usage:
start = pd.Timestamp("2022-07-01").normalize()
end = pd.Timestamp.now().normalize()
market=[#"CAISO", 
    "PJM", 
    "NYISO"]
for m in market:
    download_market_data(m, start, end)


In [ ]:
caiso = gridstatus.CAISO()
start = pd.Timestamp("Jul 1, 2022").normalize()
end = pd.Timestamp.now().normalize()

locations = ["TH_NP15_GEN-APND", "TH_SP15_GEN-APND", "TH_ZP26_GEN-APND"]

lmp_df = caiso.get_lmp(
    start=start, end=end, market="DAY_AHEAD_HOURLY", locations=locations, sleep=5
)

  0%|          | 0/39 [00:00<?, ?it/s]2025-10-05 22:44:25 - DEBUG - Dataset config: {'query': {'path': 'SingleZip', 'resultformat': 6, 'queryname': 'PRC_LMP', 'version': 12}, 'params': {'market_run_id': 'DAM', 'node': None, 'grp_type': [None, 'ALL', 'ALL_APNODES']}}
2025-10-05 22:44:25 - INFO - Fetching URL: http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&market_run_id=DAM&node=TH_NP15_GEN-APND,TH_SP15_GEN-APND,TH_ZP26_GEN-APND&startdatetime=20220701T07:00-0000&enddatetime=20220801T07:00-0000
2025-10-05 22:45:43 - DEBUG - Found 1 files: ['20220701_20220801_PRC_LMP_DAM_20251005_22_44_25_v12.csv']
2025-10-05 22:45:43 - DEBUG - Parsing file: 20220701_20220801_PRC_LMP_DAM_20251005_22_44_25_v12.csv
  3%|▎         | 1/39 [01:23<53:05, 83.83s/it]2025-10-05 22:45:48 - DEBUG - Dataset config: {'query': {'path': 'SingleZip', 'resultformat': 6, 'queryname': 'PRC_LMP', 'version': 12}, 'params': {'market_run_id': 'DAM', 'node': None, 'grp_type': [None, 'ALL', '